In [31]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [32]:
api_key = os.getenv('ANTHROPIC_API_KEY')

In [33]:
from autogen_ext.models.anthropic import AnthropicChatCompletionClient
model_client = AnthropicChatCompletionClient(model = "claude-sonnet-4-5-20250929",api_key=api_key)

In [34]:
from autogen_agentchat.messages import TextMessage
message = TextMessage(content = "I'd like to go to London", source="user")
message

TextMessage(id='5ae0ef08-496b-4869-b764-9ffa3e71c685', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 11, 20, 16, 1, 24, 447878, tzinfo=datetime.timezone.utc), content="I'd like to go to London", type='TextMessage')

In [35]:
from autogen_agentchat.agents import AssistantAgent

agent = AssistantAgent(
    name = "Airline_agent",
    model_client=model_client,
    system_message="You are a helpful assistant for an airline. You give short, humorous answers",
    model_client_stream=True
)

In [36]:
from autogen_core import CancellationToken

response = await agent.on_messages([message], cancellation_token=CancellationToken())
response.chat_message.content

'Excellent choice! London is brilliant - tea, red buses, and people apologizing to *you* when you bump into *them*. \n\nWhen are you thinking of traveling? And where will you be flying from?'

In [37]:
import sqlite3

if os.path.exists("tickets1.db"):
    os.remove("tickets1.db")

conn = sqlite3.connect("tickets1.db")
c = conn.cursor()
c.execute("CREATE TABLE cities (city_name TEXT PRIMARY_KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [38]:
def save_city_price(city_name, round_trip_price):
    conn = sqlite3.connect("tickets1.db")
    c = conn.cursor()
    c.execute("REPLACE INTO CITIES (city_name, round_trip_price) VALUES (?,?)", (city_name.lower(), round_trip_price))
    conn.commit()
    conn.close()

In [40]:
save_city_price("London", 299)
save_city_price("Paris", 499)
save_city_price("Rome", 559)
save_city_price("Sydney", 799)
save_city_price("Berlin", 399)

In [41]:
def get_city_price(city_name: str) -> float | None:
    """Get the round trip ticket price to the city"""
    conn = sqlite3.connect("tickets1.db")
    c = conn.cursor()
    c.execute("SELECT round_trip_price from cities WHERE city_name = ?", (city_name.lower(),))
    result = c.fetchone()
    conn.close()
    return result[0] if result else None

In [42]:
get_city_price("Rome")

559.0

In [43]:
from autogen_agentchat.agents import AssistantAgent

smart_agent = AssistantAgent(
    name = "smart_airline_agent",
    model_client = model_client, 
    system_message = "You are a helpful assistant for an airline. You give short and humorous answers including the roundtrip price of flights",
    model_client_stream=True,
    tools = [get_city_price],
    reflect_on_tool_use=True
)

In [44]:
response = await smart_agent.on_messages([message], cancellation_token=CancellationToken())
for inner_message in response.inner_messages:
    print(inner_message.content)

response.chat_message.content

[FunctionCall(id='toolu_01BB5797yTUwVYeUR25Rhpes', arguments='{"city_name": "London"}', name='get_city_price')]
[FunctionExecutionResult(content='299.0', name='get_city_price', call_id='toolu_01BB5797yTUwVYeUR25Rhpes', is_error=False)]


"Excellent choice! A roundtrip ticket to London will cost you $299. That's a small price to pay for tea, crumpets, and the chance to perfect your British accent. Mind the gap (in your wallet)! ☕🇬🇧"